In [ ]:
import numpy as np
import pandas as pd
import ortools
from ortools.linear_solver import pywraplp
import itertools

In [ ]:
# define fixed puzzle values
initialBoard = np.array(pd.read_csv('initialBoard.csv',header=None))

In [ ]:
# declare solver
solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# decision variables
# row i, column j, value k
x = np.zeros((9,9,9)).astype('object')
for i in range(9):
    for j in range(9):
        for k in range(9):
            if initialBoard[i,j] == 0:
                x[i,j,k] = solver.IntVar(0, 1, 'x'+str(i)+str(j)+str(k))
            else:
                x[i,j,k] = int(initialBoard[i,j] == k + 1)

# constraint for no row repeats
for j in range(9):
    for k in range(9):
        solver.Add(np.sum(x[:,j,k]) == 1)

# constraint to ensure all numbers used
for i in range(9):
    for j in range(9):
        solver.Add(np.sum(x[i,j,:]) == 1)

# constraint for no column repeats
for i in range(9):
    for k in range(9):
        solver.Add(np.sum(x[i,:,k]) == 1)
# no cell repeats
for k in range(9):
    for p in range(1,4):
        for q in range(1,4):
            io=(3*q-2)-1
            ie=(3*q)
            jo=(3*p-2)-1
            je=(3*p)
            val_sum = np.sum(x[io:ie,jo:je,k])
            solver.Add(val_sum==1)
# arbitrary objective
solver.Minimize(np.sum(x))

In [ ]:
"""Solve the problem"""
result_status = solver.Solve()

In [ ]:
# pull the results from or-tools objects
finalBoard = np.zeros((9,9)).astype('object')
x_result=np.zeros((9,9,9))
for i in range(9):
    for j in range(9):
        for k in range(9):
            if isinstance(x[i,j,k],int):
                x_result[i,j,k] = x[i,j,k]*(k+1)
            else:
                x_result[i,j,k] = x[i,j,k].solution_value()*(k+1)
finalBoard = np.sum(x_result,axis=2)

In [ ]:
# print solution to file
df = pd.DataFrame(finalBoard)
df.to_csv("finalBoard.csv",index=False,header=False)